In [2]:
import bs4
import csv
import enchant
import re
from difflib import SequenceMatcher
from selenium import webdriver

path_to_webdriver = '/Users/ellykuhlman/Downloads/chromedriver'
browser = webdriver.Chrome(executable_path = path_to_webdriver)

url = 'https://www.socialhourcocktails.com/recipes'
browser.get(url)

html = browser.page_source

In [3]:
html = html.replace('\xa0', ' ').\
            replace('\u200b', '').\
            replace('Go to recipe page...', '').\
            replace('Go recipe page...', '').\
            replace('Go to recipe page…', '').\
            replace('Continued on recipe page..', '').\
            replace(' Panel 2', '').\
            replace(' Panel 3', '')

soup = bs4.BeautifulSoup(html, "html5lib")

In [4]:
d = enchant.Dict('en_US')

In [29]:
one_recipe_soup = soup.find_all('div', class_='style-iwqinworimageItemzoom')
two_recipe_soup = soup.find_all('div', class_='mg2imageItemimageWrapper')

recipe_soup = one_recipe_soup + two_recipe_soup

recipes = {}
ignore = ['demerara', 'Campari', 'Angostura']

for recipe in recipe_soup:
    name = recipe.find(itemprop='name').contents[0]
    ingredient_soup = recipe.find(itemprop="description").find_all("span")
    instructions = [i.contents[0] for i in ingredient_soup \
                     if i.contents and i.contents[0] != ' ']
    
    if len(instructions) > 1 and instructions[-2][0].isupper():
        instructions[-2] = instructions[-2] + instructions[-1]
        instructions.pop()
    
    ingredients = [i.replace('oz', '').\
                   replace('cups', '').\
                   replace('cup', '').\
                   replace('dashes', '').\
                   replace('dash', '').\
                   replace('teaspoons', '').\
                   replace('teaspoon', '').\
                   replace('teapsoon', '').\
                   replace('tablespoons', '').\
                   replace('tablespoon', '').\
                   replace('gallon', '').\
                   replace('scant', '').\
                   replace('small', '').\
                   replace('ml bottle', '').\
                   replace('bottles', '')\
                   for i in instructions[:-1]]
    
    for i in range(len(ingredients)):
        ingredients[i] = re.sub('[^\\sA-Za-z]', '', ingredients[i]).strip()
        
        if 'simple syrup' in ingredients[i]:
            ingredients[i] = 'simple syrup'
        if 'bourbon' in ingredients[i] and 'rye' in ingredients[i]:
            ingredients[i] = 'whiskey'
        if 'optional' in ingredients[i]:
            ingredients[i] = ingredients[i].replace('optional', '*')
        
        if len(re.findall('\S+', ingredients[i])) > 2:
            answer = input('Changes to {}: '.format(ingredients[i]))
            if answer != '':
                ingredients[i] = answer
            else:
                continue
        
        for word in re.findall('\S+', ingredients[i]):
            if d.check(word) == False and word not in ignore:
                answer = input('replace {} in {}: '.format(word, ingredients[i]))
                if answer != '':
                    ingredients[i].replace(word, answer)
                else:
                    ignore.append(word)
    
    recipes[name] = ingredients, instructions

        

Changes to baking spice aromatic bitters: 
Changes to Campari or   Aperol: Campari
Changes to ros or brut sparkling wine: sparkling wine
Changes to pinch salt *: salt*
replace salt* in salt*: 
Changes to half a lemon quartered: lemon
replace blanco in blanco tequila: 
Changes to about  of an egg white: egg white
Changes to fresh pineapple juice or   chunks pineapple: pineapple or pineapple juice
replace prosecco in prosecco: 
replace Lpez in Coco Lpez: Lopez
Changes to peach brandy or apple brandy: apple brandy
Changes to quart cold water: water
replace teainfused in teainfused bourbon: tea-infused
Changes to tsp demerara syrup  or other sugar: demerara syrup
Changes to orange bitters *: 
Changes to Jerry Thomas Decanter Bitters *: 
Changes to of orange bitters *: orange bitters *
replace * in orange bitters *: 
Changes to lime disc *: lime
Changes to Cointreau  or other orange liqueur: orange liqueur
Changes to egg white  *: 
Changes to mint leaves no stems: 
Changes to chilled soda w

In [34]:
def check_sim(check_item, check_list):
    for element in check_list:
        if SequenceMatcher(None, check_item, element).ratio() > 0.625:
            answer = input('is {} the same as {}? '.format(check_item, element))
            if answer == 'y':
                return element
    return None

In [41]:
master_ingredients = []

for key in recipes:
    for i in range(len(recipes[key][0])):
        if '*' in recipes[key][0][i]:
            continue
        if recipes[key][0][i] == '' or recipes[key][0][i] == ' ':
            continue
        if recipes[key][0][i] not in master_ingredients:
            replacement = check_sim(recipes[key][0][i], master_ingredients)
            if replacement == None:
                master_ingredients.append(recipes[key][0][i])
            else:
                recipes[key][0][i] = replacement
                


is simple syrup the same as maple syrup? n
is lime juice the same as lemon juice? n
is peach liqueur the same as coffee liqueur? n
is lemon shrub the same as lemon juice? n
is dry vermouth the same as sweet vermouth? n
is orange liqueur the same as coffee liqueur? n
is orange liqueur the same as peach liqueur? n
is soda water the same as water? n
is orange bitters the same as Angostura bitters? n
is orange bitters the same as orange liqueur? n
is cinnamon sticks the same as cinnamon syrup? n
is whole nutmeg the same as grated nutmeg? n
is orange zest the same as orange liqueur? n
is orange zest the same as orange bitters? n
is ginger the same as gin? n
is ginger the same as ginger beer? n
is hazelnut-infused bourbon the same as teainfused bourbon? n


In [42]:
with open('master_cocktails.csv', 'w', newline='') as f:
    cocktailWriter = csv.writer(f)
    
    for key in recipes.keys():
        cocktailWriter.writerow((str(key), str(recipes[key][0]), str(recipes[key][1])))
        

In [43]:
with open('master_bar.csv', 'w', newline='') as g:
    barWriter = csv.writer(g)
    
    barWriter.writerow(master_ingredients)